In [6]:
from pathlib import Path
import collections
import re
import plotly.graph_objects as go
import json
print("???")

???


In [26]:
print(f'current path: {Path().absolute()}')       

functions=[
    "base",
    "latinize",
    "phonetic-latinize",
    "add-diacritic",
    "delete-letters",
    "permute-letters",
    "introduce-extra-letters",
    "confuse-letters",
    "sample-substitute",
    "remove-punctuation",
    "add-comma",
    #"add-punctuation"
]

def transpose(l):
    return list(map(list, zip(*l)))

def get_model_names(lang):
    single = [f'{lang}_{f}' for f in functions]
    excl = [f'{lang}_excl-{f}' for f in functions]
    excl.remove('lv_excl-base')
    # excl.remove('lv_excl-delete-letters')
    # excl.remove('lv_excl-introduce-extra-letters')
    # excl.remove('lv_excl-add-comma')
    # return single + excl
    return single

def build_table(validation_file, lang, metric):
    table = dict((n, {}) for n in get_model_names(lang))
    exp_path = Path() / 'experiments'

    for f_exp in get_model_names(lang):
        exp = exp_path / f_exp
        for f_eval in functions:
            report_file = exp / 'test' / f'{validation_file}.{f_eval}.{metric}'
            if not report_file.exists():
                table[f_exp][f_eval] = '-'
            else:
                text = report_file.read_text()
                match = re.search(r'([0-9]*\.)?[0-9]+', text)
                table[f_exp][f_eval] = match[0]
        if metric == 'ter':
            file = exp / 'test' / 'overall.ter'
            if file.exists():
                ter = re.search(r'([0-9]*\.)?[0-9]+', file.read_text())[0]
                table[f_exp]['average'] = ter
            else:
                table[f_exp]['average'] = '-'
        if metric == 'bleu':
            if any(x == '-' for x in table[f_exp].values()):
                table[f_exp]['average'] = '-'
                continue
            average = sum(map(lambda x: float(x), table[f_exp].values())) / len(table[f_exp].values())
            table[f_exp]['average'] = round(average, 2)

    return table

bleu_table = build_table('newsdev2017.tc', 'lv', 'bleu')
ter_table = build_table('newsdev2017.tc', 'lv', 'ter')
with open('out2.txt', 'w') as f:
    f.write(json.dumps(ter_table))
print(json.dumps(ter_table))

current path: /home/TILDE.LV/arturs.stafanovics/robust_NMT
{"lv_base": {"base": "0.7235", "latinize": "0.9267", "phonetic-latinize": "0.9457", "add-diacritic": "0.7544", "delete-letters": "0.7485", "permute-letters": "0.7506", "introduce-extra-letters": "0.7484", "confuse-letters": "0.7452", "sample-substitute": "0.7450", "remove-punctuation": "0.9159", "add-comma": "0.7269", "average": "0.7897"}, "lv_latinize": {"base": "0.7062", "latinize": "0.7153", "phonetic-latinize": "0.7799", "add-diacritic": "0.7353", "delete-letters": "0.7304", "permute-letters": "0.7354", "introduce-extra-letters": "0.7236", "confuse-letters": "0.7384", "sample-substitute": "0.7305", "remove-punctuation": "0.8970", "add-comma": "0.7088", "average": "0.7435"}, "lv_phonetic-latinize": {"base": "0.7141", "latinize": "0.8039", "phonetic-latinize": "0.7231", "add-diacritic": "0.7341", "delete-letters": "0.7400", "permute-letters": "0.7403", "introduce-extra-letters": "0.7310", "confuse-letters": "0.7409", "sample-

In [4]:
values = [list(ter_table[n].values()) for n in get_model_names('lv')]
values = transpose(values)
print(len(values))

fig = go.Figure(data = go.Table(
    header=dict(values=['model\data'] + functions + ['average']),
    cells=dict(values=[get_model_names('lv')] + values)))

fig.update_layout(
    autosize=False,
    width=2050,
    height=1000,
)
fig.show()

9


In [17]:
values = [list(bleu_table[n].values()) for n in get_model_names('lv')]
values = transpose(values)
print(len(values))

fig = go.Figure(data = go.Table(
    header=dict(values=['model\data'] + functions + ['average']),
    cells=dict(values=[get_model_names('lv')] + values)))

fig.update_layout(
    autosize=False,
    width=2050,
    height=1000,
)
fig.show()


9
